In [285]:
# Support Vector Machine
import pandas as pds
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [286]:
dataset = pds.read_csv("league_of_lol.csv")

# insert on a dataframe called pd the following columns: blueKills', 'blueAvgLevel', 'blueWins'
pd = dataset[['blueKills', 'blueAvgLevel', 'blueFirstBlood', 'blueAssists', 'blueDeaths','blueWins']]
pd.head()

X = pd.iloc[:, :-1].values
y = pd.iloc[:, -1].values 

In [287]:
def calcularEntropia(data):
    clases, numeroClases = np.unique(data, return_counts=True)
    entropia = np.sum([(-numeroClases[i]/np.sum(numeroClases))*np.log2(numeroClases[i]/np.sum(numeroClases)) for i in range(len(clases))])
    return entropia

In [288]:
def calcularGanancia(data, atributo, clase):
    entropiaDataSet = calcularEntropia(data[clase])
    valores, numeroValores = np.unique(data[atributo], return_counts=True)
    pesoEntropia = np.sum([(numeroValores[i]/np.sum(numeroValores))*calcularEntropia
                           (data.where(data[atributo]==valores[i]).dropna()[clase]) for i in range(len(valores))])
    ganancia = entropiaDataSet - pesoEntropia
    return ganancia

In [289]:
# Set the features and label
features = pd.columns[:-1]
label = pd.columns[-1]
parent=None
features

Index(['blueKills', 'blueAvgLevel', 'blueFirstBlood', 'blueAssists',
       'blueDeaths'],
      dtype='object')

In [290]:
def crearArbol(dataset, df, features,label, parent):
    datum = np.unique(dataset[label], return_counts=True)
    unique_data = np.unique(dataset[label])
    
    if len(unique_data) <= 1:
        return unique_data[0]
    elif len(dataset)==0:
        return unique_data[np.argmax(datum[1])]
    elif len(features)==0:
        return parent
    
    else:
        parent = unique_data[np.argmax(datum[1])]
        item_values = [calcularGanancia(dataset, feature, label) for feature in features]
        mejor_ganancia_index = np.argmax(item_values)
        mejor_ganancia = features[mejor_ganancia_index]
        tree = {mejor_ganancia:{}}
        features = [i for i in features if i != mejor_ganancia]
        
        for value in np.unique(dataset[mejor_ganancia]):
            sub_data = dataset.where(dataset[mejor_ganancia]==value).dropna()
            sub_tree = crearArbol(sub_data, df, features, label, parent)
            tree[mejor_ganancia][value] = sub_tree
        return(tree)

In [291]:
#Dividir el conjunto de datos en entrenamiento y prueba, tomando el 80% de los datos para entrenamiento y el 20% para prueba
dataFrameEntreno = pds.DataFrame()

dataFrameEntreno["blueKills"] = pd.iloc[0:int((len(pd)*0.8)), 0]
dataFrameEntreno["blueAvgLevel"] = pd.iloc[0:int((len(pd)*0.8)), 1]
dataFrameEntreno["blueWins"] = pd.iloc[0:int((len(pd)*0.8)), 2]
dataFrameEntreno["blueFirstBlood"] = pd.iloc[0:int((len(pd)*0.8)), 3]
dataFrameEntreno["blueAssists"] = pd.iloc[0:int((len(pd)*0.8)), 4]
dataFrameEntreno["blueDeaths"] = pd.iloc[0:int((len(pd)*0.8)), 5]
dataFrameEntreno.head()


,blueKills,blueAvgLevel,blueWins,blueFirstBlood,blueAssists,blueDeaths
0,9,6.6,1,11,6,0
1,5,6.6,0,5,5,0
2,7,6.4,0,4,11,0
3,4,7.0,0,5,5,0
4,6,7.0,0,6,6,0


In [292]:
dataFrameValidacion = pds.DataFrame()

dataFrameValidacion["blueKills"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 0]
dataFrameValidacion["blueAvgLevel"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 1]
dataFrameValidacion["blueWins"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 2]
dataFrameValidacion["blueFirstBlood"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 3]
dataFrameValidacion["blueAssists"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 4]
dataFrameValidacion["blueDeaths"] = pd.iloc[int((len(pd)*0.8)):int((len(pd)*0.9)), 5]
dataFrameValidacion.head()


,blueKills,blueAvgLevel,blueWins,blueFirstBlood,blueAssists,blueDeaths
7903,6,6.4,0,12,7,0
7904,12,7.4,1,6,3,1
7905,9,7.2,0,11,3,1
7906,1,6.6,1,1,6,0
7907,6,7.2,1,8,7,0


In [293]:
dataFrameTest = pds.DataFrame()

dataFrameTest["blueKills"] = pd.iloc[int((len(pd)*0.9)):-1, 0]
dataFrameTest["blueAvgLevel"] = pd.iloc[int((len(pd)*0.9)):-1, 1]
dataFrameTest["blueWins"] = pd.iloc[int((len(pd)*0.9)):-1, 2]
dataFrameTest["blueFirstBlood"] = pd.iloc[int((len(pd)*0.9)):-1, 3]
dataFrameTest["blueAssists"] = pd.iloc[int((len(pd)*0.9)):-1, 4]
dataFrameTest["blueDeaths"] = pd.iloc[int((len(pd)*0.9)):-1, 5]
dataFrameTest.head(20)



,blueKills,blueAvgLevel,blueWins,blueFirstBlood,blueAssists,blueDeaths
8891,3,7.0,0,0,6,0
8892,11,7.0,0,15,5,1
8893,8,7.4,0,6,5,0
8894,7,6.8,0,8,10,0
8895,3,7.0,0,4,4,0
8896,5,6.6,1,3,11,0
8897,7,7.0,1,7,6,1
8898,6,7.4,1,7,7,1
8899,13,7.6,1,11,3,1
8900,5,7.4,0,7,2,0


In [294]:
dataFrameEntreno.head(20)

,blueKills,blueAvgLevel,blueWins,blueFirstBlood,blueAssists,blueDeaths
0,9,6.6,1,11,6,0
1,5,6.6,0,5,5,0
2,7,6.4,0,4,11,0
3,4,7.0,0,5,5,0
4,6,7.0,0,6,6,0
5,5,7.0,0,6,3,1
6,7,6.8,1,7,6,1
7,5,6.4,0,3,13,0
8,7,7.2,0,8,7,0
9,4,6.8,1,5,5,1


In [295]:
# Train the decision tree model
decision_tree = crearArbol(dataFrameEntreno,dataFrameEntreno,features,label,parent)
print(decision_tree)

{'blueKills': {0: 0.0, 1: {'blueAssists': {0.0: 1.0, 1.0: {'blueDeaths': {0.0: {'blueAvgLevel': {6.8: 0.0, 7.0: {'blueFirstBlood': {0.0: 1.0, 1.0: 0.0}}}}, 1.0: {'blueFirstBlood': {0.0: 1.0, 1.0: {'blueAvgLevel': {6.8: 0.0, 7.0: 1.0}}, 2.0: {'blueAvgLevel': {6.8: 1.0, 7.0: 1.0}}}}}}, 2.0: {'blueAvgLevel': {6.4: 0.0, 6.6: {'blueFirstBlood': {1.0: {'blueDeaths': {0.0: 0.0, 1.0: 0.0}}, 2.0: 0.0}}, 6.8: {'blueFirstBlood': {1.0: {'blueDeaths': {0.0: 0.0, 1.0: 0.0}}, 2.0: {'blueDeaths': {0.0: 0.0, 1.0: 1.0}}, 3.0: 0.0}}, 7.0: 0.0, 7.2: 0.0}}, 3.0: {'blueAvgLevel': {6.6: 0.0, 6.8: {'blueFirstBlood': {0.0: 0.0, 1.0: {'blueDeaths': {0.0: 0.0, 1.0: 0.0}}, 2.0: 0.0}}, 7.0: {'blueFirstBlood': {0.0: {'blueDeaths': {0.0: 1.0, 1.0: 0.0}}, 1.0: {'blueDeaths': {0.0: 0.0}}, 2.0: 1.0}}, 7.2: 0.0}}, 4.0: {'blueAvgLevel': {6.0: 0.0, 6.4: {'blueDeaths': {0.0: {'blueFirstBlood': {1.0: 0.0}}, 1.0: 0.0}}, 6.6: {'blueFirstBlood': {0.0: 1.0, 1.0: {'blueDeaths': {0.0: 0.0, 1.0: 0.0}}, 2.0: 1.0}}, 6.8: 0.0, 7.0: 0

In [296]:
def predict(test_data, arbol):
    for nodes in arbol.keys():
        value = test_data[nodes]
        try:
            arbol = arbol[nodes][value]
        except Exception as e:
            return 0.0
        
        prediction = 0
        
        if type(arbol) is dict:
            prediction = predict(test_data, arbol)
        else:
            prediction = arbol
            break;
    return prediction

In [297]:
sampleData = {'blueKills': 1, 'blueAvgLevel': 13, 'blueFirstBlood': 0, 'blueAssists': 8, 'blueDeaths': 6}
sampleData = pds.Series(sampleData)
print(predict(sampleData, decision_tree))

0.0


In [302]:
# Predict using the trained model
# sample_data = {'glucose':86,'bloodpressure':104}

sample_data = dataFrameTest.iloc[:, 0:5]
newSampleData = {}
newSampleData["blueKills"] = 1
newSampleData["blueAvgLevel"] = 1
print(newSampleData)
resultados = []
for index, values in sample_data.iterrows():
    newSampleData["blueKills"] = values[0]
    newSampleData["blueAvgLevel"] = values[1]
    newSampleData["blueFirstBlood"] = values[2]
    newSampleData["blueAssists"] = values[3]
    newSampleData["blueDeaths"] = values[4]
    newSampleData = pds.Series(newSampleData)
    resultados.append(predict(newSampleData,decision_tree))
    
for i in range(len(resultados)):
    if resultados[i] == 1:
        print("Gano el equipo azul")
    else:
        print("Gano el equipo rojo")


{'blueKills': 1, 'blueAvgLevel': 1}
Gano el equipo azul
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo azul
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo azul
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo azul
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo azul
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo rojo
Gano el equipo azul
Gano el equipo rojo
Gano

In [303]:
# Medir el rendimiento del modelo

# Accuracy
accuracy = accuracy_score(dataFrameTest["blueWins"], resultados)
print("Accuracy: ", accuracy)


Accuracy:  0.5096251266464032


In [304]:
# Medir el rendimiento del modelo

# Accuracy
accuracy = accuracy_score(dataFrameValidacion.iloc[0:-1, 2], resultados)
print("Accuracy: ", accuracy)


Accuracy:  0.47922998986828774
